In [1]:
import pandas as pd
import requests as rq
import json
from datetime import datetime as dt
import ast

In [14]:
api = rq.get("https://api.cartolafc.globo.com/atletas/mercado")

In [15]:
market = json.loads(api.text)

In [16]:
market.keys()

dict_keys(['atletas', 'clubes', 'posicoes', 'status'])

In [17]:
round_num = '10'

In [18]:
roundN = pd.DataFrame(market["atletas"])

In [19]:
with open("../../status_id.json") as file:
    status_id = json.load(file)
with open("../../positions.json") as file:
    pos_id = json.load(file)

In [8]:
roundN = roundN[["rodada_id", "atleta_id", "nome", "apelido", "posicao_id", "clube_id", "scout", "pontos_num", "preco_num", "variacao_num"]]
roundN.columns = ["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "scout", "total_score", "current_price", "price_delta"]

In [9]:
roundN.position = roundN.position.apply(lambda x: pos_id[str(x)])

In [10]:
def scout_parser(scout, col_names):
    scout_dict = ast.literal_eval(scout)
    scout_list = []
    for col in col_names:
        try:
            el = scout_dict[col]
        except:
            el = 0
        scout_list.append(el)
    return pd.DataFrame(columns = col_names, data = [scout_list]).iloc[0]

In [11]:
col_names = ["G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC"]
roundN[col_names] = roundN.scout.apply(lambda x: scout_parser(str(x), col_names))
roundN = roundN[["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", "total_score", "current_price", "price_delta"]]

In [12]:
roundN.head(10)

,round_num,player_id,player_name,player_nickname,position,team_id,G,A,FT,FD,...,DD,RB,GC,CV,CA,GS,FC,total_score,current_price,price_delta
0,9,86757,Everton Sousa Soares,Everton,ATA,284,3,0,0,3,...,0,8,0,0,1,0,0,0.00,17.31,0.00
1,9,38509,Diego Alves Carreira,Diego Alves,GOL,262,0,0,0,0,...,9,0,0,0,0,5,0,0.00,7.78,0.00
2,9,83257,Gabriel Barbosa Almeida,Gabriel,ATA,262,5,0,0,8,...,0,4,0,0,2,0,4,14.30,18.25,2.46
3,9,80853,Pedro Tonon Geromel,Pedro Geromel,ZAG,284,0,0,0,1,...,0,23,0,0,2,0,3,0.00,14.75,0.00
4,9,60819,Anderson Vital da Silva,Dedé,ZAG,283,2,0,0,4,...,0,16,0,0,2,0,8,-1.40,14.72,-2.13
5,9,87342,Walter Kannemann,Kannemann,ZAG,284,1,0,0,0,...,0,5,0,0,3,0,9,0.00,14.70,0.00
6,9,42500,Fagner Conserva Lemos,Fagner,LAT,264,0,2,0,1,...,0,14,0,0,1,0,1,0.00,15.29,0.00
7,9,70800,Jorge Sampaoli,Jorge Sampaoli,TEC,277,0,0,0,0,...,0,0,0,0,0,0,0,6.12,12.18,0.20
8,9,78445,Renê Rodrigues Martins,Renê,LAT,262,0,1,0,1,...,0,11,0,0,1,0,2,5.00,7.82,-0.30
9,9,42234,Cássio Ramos,Cássio,GOL,264,0,0,0,0,...,8,0,0,0,2,4,0,0.00,8.71,0.00


In [13]:
roundN.to_csv("2019_round"+ round_num + "_scouts.csv", index=False)

In [20]:
status = roundN[["rodada_id", "atleta_id", "status_id"]]

In [21]:
status.rodada_id = status.rodada_id + 1
now = dt.now()
status["query_time"] = dt(now.year, now.month, now.day, now.hour, now.minute, now.second)
status.status_id = status.status_id.apply(lambda x: status_id[str(x)])

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
status = status[["rodada_id", "query_time", "atleta_id", "status_id"]]
status.columns = ["round_num", "query_time", "player_id", "status_id"]
status.head()

,round_num,query_time,player_id,status_id
0,10,2019-07-13 14:14:27,61211,Nulo
1,10,2019-07-13 14:14:27,87342,Nulo
2,10,2019-07-13 14:14:27,38509,Provável
3,10,2019-07-13 14:14:27,93893,Provável
4,10,2019-07-13 14:14:27,86842,Nulo


In [23]:
filename = "../status/2019_round"
if status.round_num[0] < 10:
    filename = filename + "0" + str(status.round_num[0]) + "_status.csv"
else:
    filename = filename + str(status.round_num[0]) + "_status.csv"

status.to_csv(filename, index=False)